In [ ]:
from fhd import *
from matplotlib import pyplot as plt
import numpy as np
import kaveh.toolbox as kx

In [ ]:
import matplotlib as mpl
mpl.rcParams['xtick.color'] = 'gray'
mpl.rcParams['ytick.color'] = 'gray'
mpl.rcParams['axes.labelcolor'] = 'gray'

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# helper functions
def load_fhd_for_fft(f_name):
    fhd_content = load(f_name)
    data = np.squeeze(fhd_content['channel_{}_voltage'.format(0)].T)
    if data.size % 2 != 0:
        data = data[0:-1]
    return data

    


In [ ]:
# data_without_mpm = load_fhd_for_fft('/mnt/data/temp/2018_10_25/standalone_ephys_171025.fhd') # without mu metal, with shield, no mpm
# data_with_mpm = load_fhd_for_fft('/mnt/data/temp/2018_10_25/standalone_ephys_170920.fhd') # without mu metal, with shield, with mpm

data_with_mpm = load_fhd_for_fft('/mnt/data/temp/2018_10_25/standalone_ephys_170741.fhd') # with mu metal, with shield, with mpm
data_without_mpm = load_fhd_for_fft('/mnt/data/temp/2018_10_25/standalone_ephys_170603.fhd') # with mu metal, with shield, without mpm

Fs = 15000.0
dt = 1.0/(Fs)

fig_size = (15,5)



In [ ]:
filtered_wo = kx.notch_all_harmonics(data_without_mpm, 60.0, 15000.0)
filtered_wo = kx.notch_all_harmonics(filtered_wo, 100.2, 15000.0)
plt.figure(figsize=fig_size)
plt.plot(np.arange(0, data_without_mpm.size)*dt, filtered_wo)

filtered_wo = kx.butter_lowpass_filter(filtered_wo, 1000.0, 15000.0)
plt.figure(figsize=fig_size)
plt.plot(np.arange(0, data_without_mpm.size)*dt, filtered_wo)


# filtered_wo = kx.butter_bandpass_filter(data_without_mpm, 300.0, 3000.0, 15000.0)
# plt.figure(figsize=fig_size)
# plt.plot(np.arange(0, data_without_mpm.size)*dt, filtered_wo)


plt.figure(figsize=fig_size)
plt.plot(np.arange(0, data_without_mpm.size)*dt, data_without_mpm)


In [ ]:
# filtered_w = kx.notch_all_harmonics(data_with_mpm, 60.0, 15000.0)
# filtered_w = kx.notch_all_harmonics(filtered_w, 100.2, 15000.0)
# plt.figure(figsize=fig_size)
# plt.plot(np.arange(0, data_with_mpm.size)*dt, filtered_w)

# filtered_w = kx.butter_lowpass_filter(filtered_w, 1000.0, 15000.0)
# plt.figure(figsize=fig_size)
# plt.plot(np.arange(0, data_with_mpm.size)*dt, filtered_w)


filtered_w = kx.butter_bandpass_filter(data_with_mpm, 300.0, 3000.0, 15000.0)
plt.figure(figsize=fig_size)
plt.plot(np.arange(0, data_with_mpm.size)*dt, filtered_w)


plt.figure(figsize=fig_size)
plt.plot(np.arange(0, data_with_mpm.size)*dt, data_with_mpm)


In [ ]:
# Threshold, align, and PCA
from sklearn.decomposition import PCA

signal = filtered_w

thresh = 150.0

spike_indices = np.squeeze(np.where(signal > thresh))
peak_times,_ = scipy.signal.find_peaks(signal[spike_indices])

spike_indices = spike_indices[peak_times]
plt.figure(figsize=fig_size)
# plt.plot(np.arange(0, signal.size)*dt, signal)
plt.plot(signal)
plt.vlines(spike_indices, np.min(signal), np.max(signal), 'r', zorder=10, alpha=0.25)

pre_window = 0.003
post_window = 0.01
signal_size = signal.size
to_exclude = []

pre_index = int(np.round(pre_window / dt))
post_index = int(np.round(post_window / dt))
aligned_spikes = np.array([signal[i - pre_index : i + post_index ] 
                for i in spike_indices if i not in to_exclude if (i + post_index) < signal_size
                    if (i - pre_index) >= 0])



pca = PCA(n_components=2)
pca.fit(aligned_spikes - np.mean(aligned_spikes, axis=0))

x_pca = pca.transform(aligned_spikes)

In [ ]:
print(aligned_spikes.shape)
print(700000*dt)

In [ ]:
plt.figure()

for row in aligned_spikes:
    plt.plot(np.arange(0, row.size)*dt, row, 'k', alpha =0.2, zorder=5)
    
plt.plot(np.arange(0, row.size)*dt, np.mean(aligned_spikes, axis = 0), 'r')

In [ ]:


plt.figure()
plt.plot(x_pca[:,0], x_pca[:,1], '.')